In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
ORIGIN_INPUT_DATASET = 'Food-11'
BASE_PATH = '/content/drive/MyDrive/violence_dataset'

TRAIN ='training'
TEST = 'evaluation'
VAL = 'validation'


CLASSES = ['non_violence', 'violence']

BATCH_SIZE = 32

LE_PATH = os.path.sep.join(['output', 'le.pickle'])

BASE_CSV_PATH='output'

MODEL_PATH = os.path.sep.join(['/content/drive/MyDrive/violence_dataset', 'food.model'])

UNFROZEN_PLOT_PATH = os.path.sep.join(['output','unfrozen.png'])

WARMUP_PLOT_PATH = os.path.sep.join(['output', 'warmup.png'])

In [ ]:
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
def plot_training(H, N, plotPath):
	# construct a plot that plots and saves the training history
	plt.style.use("ggplot")
	plt.figure()
	plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
	plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
	plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
	plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
	plt.title("Training Loss and Accuracy")
	plt.xlabel("Epoch #")
	plt.ylabel("Loss/Accuracy")
	plt.legend(loc="lower left")
	plt.savefig(plotPath)

In [ ]:
trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest", validation_split=0.2)

mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean

In [ ]:
trainGen = trainAug.flow_from_directory(
	'/content/drive/MyDrive/violence_dataset',
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=True,
	batch_size=BATCH_SIZE, subset='training')

valGen = trainAug.flow_from_directory(
	'/content/drive/MyDrive/violence_dataset',
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=BATCH_SIZE, subset='validation')

Found 8859 images belonging to 2 classes.
Found 2214 images belonging to 2 classes.


In [ ]:
valGen.labels

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [ ]:
import tensorflow as tf
with tf.device('/gpu:0'):
  baseModel = VGG16(weights="imagenet", include_top=False,
                    input_tensor=Input(shape=(224, 224, 3)))
  # construct the head of the model that will be placed on top of the
  # the base model
  headModel = baseModel.output
  headModel = Flatten(name="flatten")(headModel)
  headModel = Dense(512, activation="relu")(headModel)
  headModel = Dropout(0.5)(headModel)
  headModel = Dense(len(CLASSES), activation="softmax")(headModel)
  # place the head FC model on top of the base model (this will become
  # the actual model we will train)
  model = Model(inputs=baseModel.input, outputs=headModel)

58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


In [ ]:
print("[INFO] training head...")
H = model.fit(
	x=trainGen,
	steps_per_epoch=8859 // BATCH_SIZE,
	validation_data=valGen,
	validation_steps=2214 // BATCH_SIZE,
	epochs=50)

[INFO] training head...
Epoch 1/50
276/276 [==============================] - 2598s 9s/step - loss: 3.4468 - accuracy: 0.7092 - val_loss: 0.5779 - val_accuracy: 0.7957
Epoch 2/50
276/276 [==============================] - 174s 629ms/step - loss: 0.5524 - accuracy: 0.8048 - val_loss: 0.4189 - val_accuracy: 0.8229
Epoch 3/50
276/276 [==============================] - 173s 626ms/step - loss: 0.4086 - accuracy: 0.8321 - val_loss: 0.4181 - val_accuracy: 0.8197
Epoch 4/50
276/276 [==============================] - 173s 628ms/step - loss: 0.3781 - accuracy: 0.8435 - val_loss: 0.4337 - val_accuracy: 0.8193
Epoch 5/50
276/276 [==============================] - 173s 627ms/step - loss: 0.3455 - accuracy: 0.8537 - val_loss: 0.4033 - val_accuracy: 0.8356
Epoch 6/50
276/276 [==============================] - 173s 628ms/step - loss: 0.3223 - accuracy: 0.8644 - val_loss: 0.4011 - val_accuracy: 0.8351
Epoch 7/50
276/276 [==============================] - 173s 627ms/step - loss: 0.2848 - accuracy: 0.876

In [ ]:
model.save(MODEL_PATH, save_format="h5")

In [ ]:
print("[INFO] evaluating after fine-tuning network head...")
valGen.reset()
predIdxs = model.predict(x=valGen,
	steps=(2214 // BATCH_SIZE) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(valGen.classes, predIdxs,
	target_names=valGen.class_indices.keys()))
plot_training(H, 50, WARMUP_PLOT_PATH)

[INFO] evaluating after fine-tuning network head...
              precision    recall  f1-score   support

non_violence       0.93      0.79      0.85      1046
    violence       0.83      0.95      0.89      1168

    accuracy                           0.87      2214
   macro avg       0.88      0.87      0.87      2214
weighted avg       0.88      0.87      0.87      2214



FileNotFoundError: ignored

In [ ]:
trainGen.reset()
valGen.reset()

for layer in baseModel.layers[15:]:
	layer.trainable = True


for layer in baseModel.layers:
	print("{}: {}".format(layer, layer.trainable))

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fce6cf0b780>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fce6b5704a8>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fce6b570fd0>: False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fce6b532748>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fce6b5dd7f0>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fce6b5dd240>: False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fce6b5d4dd8>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fce6b5b4a90>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fce6b5aca20>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fce6b5ac080>: False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fce6b5b0208>: False
<tensorflow.python.keras.layers.convolutional.Conv2D

In [ ]:
print("[INFO] re-compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

print("[INFO] re-training model...")
H = model.fit(
	x=trainGen,
	steps_per_epoch=8859 // BATCH_SIZE,
	validation_data=valGen,
	validation_steps=2214 // BATCH_SIZE,
	epochs=20)

[INFO] re-compiling model...
[INFO] re-training model...
Epoch 1/20
276/276 [==============================] - 176s 637ms/step - loss: 0.1520 - accuracy: 0.9402 - val_loss: 0.3096 - val_accuracy: 0.8782
Epoch 2/20
276/276 [==============================] - 175s 636ms/step - loss: 0.1383 - accuracy: 0.9451 - val_loss: 0.3836 - val_accuracy: 0.8605
Epoch 3/20
276/276 [==============================] - 175s 636ms/step - loss: 0.1380 - accuracy: 0.9465 - val_loss: 0.3750 - val_accuracy: 0.8709
Epoch 4/20
276/276 [==============================] - 175s 636ms/step - loss: 0.1197 - accuracy: 0.9517 - val_loss: 0.2963 - val_accuracy: 0.8841
Epoch 5/20
276/276 [==============================] - 175s 635ms/step - loss: 0.1083 - accuracy: 0.9555 - val_loss: 0.3122 - val_accuracy: 0.8881
Epoch 6/20
276/276 [==============================] - 175s 634ms/step - loss: 0.1090 - accuracy: 0.9574 - val_loss: 0.2963 - val_accuracy: 0.8909
Epoch 7/20
276/276 [==============================] - 175s 634ms/st

In [ ]:
print("[INFO] evaluating after fine-tuning network...")
valGen.reset()
predIdxs = model.predict(x=valGen,
	steps=(2214 // BATCH_SIZE) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(valGen.classes, predIdxs,
	target_names=valGen.class_indices.keys()))
plot_training(H, 20, UNFROZEN_PLOT_PATH)
# serialize the model to disk
print("[INFO] serializing network...")
model.save(MODEL_PATH, save_format="h5")

[INFO] evaluating after fine-tuning network...
              precision    recall  f1-score   support

non_violence       0.95      0.83      0.88      1046
    violence       0.86      0.96      0.91      1168

    accuracy                           0.90      2214
   macro avg       0.91      0.89      0.90      2214
weighted avg       0.90      0.90      0.90      2214



FileNotFoundError: ignored